# FOAI Log Explorer

The goal of this notebook is to collect JSON data from the Wayback Machine so that we can create a dataset of JSON for Muckrock's [FOIA Log Explorer](https://www.muckrock.com/foi/logs/), which has pointers to PDF documents in the Wayback Machine.

Each FOIA request also had a detail view in the API. This was used to display a detail page, and critically has the information about documents that were published as a result of the request. These JSON API calls were not collected as part of the initial scraping work, but Internet Archive and Archive Team's work did archive these pages. For example here is the URL for FOIA tracking number `EPA-HQ-2018-006907` is:

https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2018-006907/Request

which has been archived in Wayback Machine:

https://web.archive.org/web/20230926082226/https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2018-006907/Request

In order to fetch these JSON documents we can walk through our list of snapshots that was created in the [Notebook](Notebook.ipynb) looking for these snapshots and then fetch them. Since there can be multiple snapshots of the same URL it can be helpful to ensure we get ones that have a 200 OK response, and prefer the largest response (the most data).

In [140]:
import re
import pandas

snaps = pandas.read_csv('snapshots.csv.gz', parse_dates=['timestamp'])
snaps = snaps[snaps.status_code == 200]
snaps = snaps[snaps.url.str.match(r'https://foiaonline.gov/foiaonline/api/request/publicRecords/.+/Request', re.IGNORECASE)]
snaps

,timestamp,url,status_code,mime_type,digest,length
5515667,2023-09-24 03:11:23+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/CBP-2004-000001/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,680
5515668,2023-09-24 03:11:33+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/CBP-2007-000001/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,680
5515669,2023-09-24 03:11:33+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/CBP-2007-000002/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,683
5515670,2023-09-24 03:11:33+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/CBP-2007-000003/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,679
5515671,2023-09-24 03:11:36+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/CBP-2007-000004/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,684
...,...,...,...,...,...,...
6870355,2023-09-27 20:51:25+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/STB-2023-000113/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,679
6870356,2023-09-27 20:52:10+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/STB-2023-000114/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,680
6870357,2023-09-27 20:52:15+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/STB-2023-000115/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,681
6870358,2023-09-27 20:51:48+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/STB-2023-000116/Request,200.0,application/json,EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM,683


## Multiple Snapshots

It's interesting to see how many snapshots of the same URL, in roughly the same amount of time yielded different results:

In [141]:
pandas.set_option('display.max_colwidth', None)

snaps[snaps.url == 'https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request']

,timestamp,url,status_code,mime_type,digest,length
6662416,2023-09-25 20:27:58+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,4S4KLZF7HZTO57LIE64EBFK5IOTJFO6Y,3945
6662417,2023-09-25 20:28:25+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,POE23XF7X6KMPNMGQS7XLMESXLF5Y7QF,3861
6662418,2023-09-25 20:29:00+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,R2LH2BKFE6SES6XPR74NEDLMP3ZMXVNU,3832
6662419,2023-09-25 20:30:00+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,QMK3WEQPJWJ2DII77F4BE5NIE5GPHOYA,4090
6662420,2023-09-25 20:31:20+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,TD6VFY7YY5H5HJXX5P3IWO35S34KPB25,3878
...,...,...,...,...,...,...
6662538,2023-09-25 21:53:35+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,MIIA6ZU34UX3E2Y77THQB27XV27J7J5F,3783
6662539,2023-09-25 21:54:20+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,WLS5GO3Y5ARFERU3J3WE3KE2QIUZSAIH,3641
6662540,2023-09-25 21:54:43+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,2PTRSCT7E43RXUVEMMJJQB3SEVWWFEDI,3841
6662541,2023-09-25 21:55:14+00:00,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,200.0,application/json,2AIP2H62MD6NW47QGNGWJCV5S57K62MV,2702


You can see the varying JSON responses here:

https://web.archive.org/web/20230000000000*/https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request

So it is helpful to prefer the response with the largest response (the most data). This can be returned by ordering all the snapshots by their length, grouping by URL, and fetching the first one.

In [142]:
snaps = snaps.sort_values('length', ascending=False)
snaps = snaps.groupby('url').agg(lambda row: row.iloc[0]).reset_index()

Now we can see that we only have one row for that URL, and it is the largest one (4119 bytes).

In [94]:
snaps[snaps.url == 'https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request']

,url,timestamp,mime_type,digest,length
1071550,https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request,2023-09-25 21:35:00+00:00,application/json,R5IMZMLOAVE2ENCELX7DRXVBVPE3JI6J,4119


## Duplicate Responses

One interesting thing about the API responses is that it's possible to see from the Wayback API's *digest* value that many of them are identical. So rather than requesting these many times, we can keep a cache on disk, and use that instead.

In [95]:
snaps.digest.value_counts()

digest
EAQDUJQCZOPLHJSCV2KAIQ3ILBVONSVM    1114674
WMCJY4Q7IGOVNP4C227AFW5JBJCFCIY2        174
ZSFEE774XFVXNFO2OU5A5V3T4UXDYHTH        147
VRSBAJNTXSRBP4NI5VRXFDPGEBEOZP5S         62
LNLEPICX5XOTFKK7NUXJFGD3AYJ4AD5X         50
                                     ...   
HEI5UI3XLTMH3BNIRQX46M2XCMY2HWCE          1
CRM2LJD46L3464I5PKQFCWL3L7EFTM7R          1
FB5SVIREUTRHGJDGCCARCGLYNBK3Q3I6          1
HU453HXYEUVW7TZXWZ3RR5B4VTU2NELO          1
MLOCH4VWWC6DG6F2KKBT2AT56VZSOBN3          1
Name: count, Length: 65641, dtype: int64

So when we fetch the responses it can be helpful if we don't issue requests to the API for responses we've already received. This can cut down on the time it takes to collect them. One way of achieving this is to create a disk cache to store the responses using the digest value as a key.

## Collect the Data

Now it's time to collect the publicRecords JSON data. We'll start by creating a cache for the responses.

In [119]:
import dbm

cache = dbm.open('cache', 'c')

And a function that looks up the snapshot.

In [120]:
import json
import time
import requests

http = requests.Session()
def get_response(row, sleep=0):
    # if we already know what the JSON is, return it
    if row.digest in cache:
        return json.loads(cache[row.digest])

    if sleep > 0:
        time.sleep(sleep)

    # fetch the data
    t = row.timestamp.strftime('%Y%m%d%H%M%S')
    wb_url = f'https://web.archive.org/web/{t}id_/{row.url}'

    # guard against malformed JSON coming back from Wayback Machine
    try:
        data = http.get(wb_url).json()
        cache[row.digest] = json.dumps(data)
        return data
    except json.JSONDecodeError:
        cache[row.digest] = '{}'
        return {}


Now we can try out the function with the row for `EPA-HQ-2017-001504` which was examined above. For comparison the Wayback URL should be: https://web.archive.org/web/20230925213500id_/https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request 

In [121]:
print(json.dumps(get_response(snaps[snaps.url == 'https://foiaonline.gov/foiaonline/api/request/publicRecords/EPA-HQ-2017-001504/Request'].iloc[0]), indent=2))

{
  "draw": 93,
  "recordsTotal": 99999,
  "recordsFiltered": 99999,
  "data": [
    {
      "trackingNumber": "EPA-HQ-2017-001504",
      "title": "Williams Ignacio Gas Plant 12_1_15 114 letter",
      "fileName": "Williams Ignacio Gas Plant 12_1_15 114 letter.pdf",
      "releaseType": "UR - Unredacted - Releasable to the General Public",
      "exemptions": null,
      "ex3statutes": [
        "N/A"
      ],
      "ex5subtypes": [
        "N/A"
      ],
      "keywords": null,
      "uploadedBy": "Scott Patefield",
      "uploadedByEmail": "patefield.scott@epa.gov",
      "createdDate": "03/22/2017 09:34 AM",
      "lastModifiedDate": "04/30/2019 12:25 PM",
      "fileType": "Adobe PDF Document",
      "fileFormat": null,
      "author": null,
      "addedBy": "2d6a4bcd-5eb4-471f-a687-dce963ace8af",
      "size": "0.2543",
      "recordReleaseDate": "03/28/2017 02:13 PM",
      "retentionPeriod": "6 year",
      "frequentlyRequested": null,
      "type": "Record",
      "recordId": 

Now we are ready to fetch the data! We can write each response to a JSONL file, after ensuring that the data has the `trackingNumber` in it (it only seems to be present when there are items in the `data` list). It's also helpful to put a little sleep in there to not go too fast and potentially get blocked by Internet Archive.

In [124]:
import re
import time
import gzip
from tqdm.notebook import tqdm

output = open('full_requests.jsonl', 'w')

for i, row in tqdm(snaps.iterrows(), total=len(snaps)):
    data = get_response(row, sleep=.5)
    # data can be the empty dict in situations where there was invalid JSON in the response
    if len(data) > 0:
        data['trackingNumber'] = re.search('publicRecords/(.+)/Request$', row.url, re.IGNORECASE).group(1)
        output.write(json.dumps(data) + "\n")

  0%|          | 0/1180808 [00:00<?, ?it/s]

## Analysis

So how many responses had data for documents?

In [155]:
count = data_count = doc_count = 0
for line in gzip.open('full_requests.jsonl.gz', 'rt'):
    count += 1
    request = json.loads(line)
    if len(request['data']) > 0:
        data_count += 1
        doc_count += len(request['data'])

print(f"total responses: {count} ; total with documents: {data_count} ; total documents: {doc_count}")

total responses: 1180636 ; total with documents: 65604 ; total documents: 320432


Why are there so few document details in the data we received?

One odd thing I noticed when inspecting the initial metadata for each request is that there appear to be a requests that had a disposition of full or partial grant, which didn't appear to have any documents associated with them online, at least at the time of archiving.

For example notice how there are no documents listed in the "Released Records" section for `CBP-2009-016175` even though it was partially granted?

https://web.archive.org/web/20230924035834/https://foiaonline.gov/foiaonline/action/public/submissionDetails?trackingNumber=CBP-2009-016175&type=Request

And as we would expect to see, the API response for `CBP-2009-016165` appears to have no documents

https://web.archive.org/web/20230924035835/https://foiaonline.gov/foiaonline/api/request/publicRecords/CBP-2009-016175/Request

The same is true for `DON-NAVY-2020-000182` which was a "Full Grant" request:

https://web.archive.org/web/20230926182344/https://foiaonline.gov/foiaonline/action/public/submissionDetails?trackingNumber=DON-NAVY-2020-000182&type=Request

We can use the initial set of metadata collected in [Notebook](Notebook.ipynb) to see how many fully or partially granted requests there are:

In [134]:
requests = pandas.read_json('data.jsonl.gz', lines=True)

In [135]:
requests.finalDisposition.value_counts()

finalDisposition
Partial Grant/Partial Denial                        369269
No Records                                          342133
Full Grant                                          184066
Improper FOIA Request for Other Reasons             111055
Not an Agency Record                                 42699
Duplicate Request                                    31664
Request Withdrawn                                    24905
Records Not Reasonably Described                     12636
Full Denial Based on Exemptions                      11655
Completely Reversed/Remanded                         11512
Other                                                 9614
All Records Referred to Another Agency                9440
Affirmed on Appeal                                    8162
Closed for Other Reasons                              6194
Partially Affirmed & Partially Reversed/Remanded      4064
Fee-Related Reason                                    3150
Name: count, dtype: int64

In [137]:
granted = requests[requests.finalDisposition.isin(['Full Grant', 'Partial Grant/Partial Denial'])]
len(granted)

553335

In [166]:
## Archived PDFs

How many archived FOIAOnline PDFs are there in the Wayback Machine?

Object `Machine` not found.


In [159]:
all_snaps = pandas.read_csv('snapshots.csv.gz')
pdf_snaps = all_snaps[all_snaps.url.str.match(r'.*foiaonline/api/request/downloadFile/.*')]
pdf_snaps

,timestamp,url,status_code,mime_type,digest,length
3686159,2023-09-25 10:56:09+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!!totals-joe%20did%20this%20because%20PA%20too%20lazy.xlsx/ce720261-c0e0-4987-aed0-160b21828462?x-csrf-token=5816939a-41bd-438e-aa2f-94751b82c85b,200.0,application/octet-stream,QRBEDCSKJ257YOW7L3AQBP4EFDWYAKZQ,13701
3686160,2023-09-30 06:17:37+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!%2020150225_Index%20of%20Manifest%20Collection%20B.pdf/a503656e-f029-42df-bfe9-10ec123511eb?x-csrf-token=cf96dfcc-e4a2-4a62-ae29-394a9e024cd4,200.0,application/octet-stream,ILS5DSVQRQNM4AABKL5TEDTCZ73ZFSVN,75976
3686161,2023-09-25 23:06:41+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!20170228_Sewer_Asbuilt_ToSRC%20(1).xlsx/a69fb996-9152-4089-921d-45a7adba5dc8?x-csrf-token=da32e734-23e8-431f-96c7-c5d3b59dcb1c,NaN,warc/revisit,5FWDJGLFCVXIOZOQ46G4WI5MUUP5FCF5,917
3686162,2023-09-25 22:44:19+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!20170228_Sewer_Asbuilt_ToSRC.xlsx/e7d90f47-786c-4860-910f-41329602057d?x-csrf-token=da32e734-23e8-431f-96c7-c5d3b59dcb1c,200.0,application/octet-stream,5FWDJGLFCVXIOZOQ46G4WI5MUUP5FCF5,15824
3686163,2023-09-26 20:32:37+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!FilesIndex_5_6_2019.pdf/cb3dbea2-4c77-46d4-927f-bdc89db2e55b?x-csrf-token=b997b043-77e4-462f-bf30-f78a1a606d63,200.0,application/octet-stream,OL5NQUXHN7LBPIOOV2BTBZPNM6LS2XH4,77723
...,...,...,...,...,...,...
5491403,2023-09-24 12:10:16+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web8272_h.pdf/59be36d5-29c1-4da9-8eb1-2ab82c1f6fe9?x-csrf-token=0477a5ec-66f8-4292-9472-43147ed09fc3,200.0,application/octet-stream,TZI3Z52KGNLIM6RSYVZ6E6CRO3VVNRH7,47259
5491404,2023-09-24 17:53:01+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web87.pdf/1e1bb4d9-ca7a-40dd-b3ca-0ffc0112c6b0?x-csrf-token=b51856db-e301-4214-9828-9f996a6059ae,200.0,application/octet-stream,BPMHEL6OUDYVUHTQPOSJMK3SNRBYY2HF,98558
5491405,2023-09-24 12:10:56+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web8808_h.pdf/a07bcbb6-9c0e-4753-9cb6-0b62ba0843fa?x-csrf-token=0477a5ec-66f8-4292-9472-43147ed09fc3,200.0,application/octet-stream,U7BX7SQRR53QTSIPVPJCARKQ2XZ2XWXP,37998
5491406,2023-09-24 12:09:15+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web9324_h.pdf/7d7a0524-6c07-4ef7-a044-6baab88f80da?x-csrf-token=0477a5ec-66f8-4292-9472-43147ed09fc3,200.0,application/octet-stream,JJJJMG6RL7Z4UCFBIU7MU5XXTCPZY7TH,46414


In [162]:
from urllib.parse import urlparse

def strip_token(url):
    uri = urlparse(url)
    return f"{uri.scheme}://{uri.netloc}{uri.path}"

strip_token('https://foiaonline.gov/foiaonline/api/request/downloadFile/~WRL3383.tmp/f44e1096-9fe7-4954-ba72-8cc4c67bb57c?x-csrf-token=67d51cc0-41cb-410f-bf17-08faf4c14f5e')

'https://foiaonline.gov/foiaonline/api/request/downloadFile/~WRL3383.tmp/f44e1096-9fe7-4954-ba72-8cc4c67bb57c'

In [164]:
pdf_snaps.url = pdf_snaps.url.map(strip_token)
pdf_snaps

/var/folders/9g/t4bgvmr566190l4nl87nrtkh0000gs/T/ipykernel_22428/3680254798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_snaps.url = pdf_snaps.url.map(strip_token)


,timestamp,url,status_code,mime_type,digest,length
3686159,2023-09-25 10:56:09+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!!totals-joe%20did%20this%20because%20PA%20too%20lazy.xlsx/ce720261-c0e0-4987-aed0-160b21828462,200.0,application/octet-stream,QRBEDCSKJ257YOW7L3AQBP4EFDWYAKZQ,13701
3686160,2023-09-30 06:17:37+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!%2020150225_Index%20of%20Manifest%20Collection%20B.pdf/a503656e-f029-42df-bfe9-10ec123511eb,200.0,application/octet-stream,ILS5DSVQRQNM4AABKL5TEDTCZ73ZFSVN,75976
3686161,2023-09-25 23:06:41+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!20170228_Sewer_Asbuilt_ToSRC%20(1).xlsx/a69fb996-9152-4089-921d-45a7adba5dc8,NaN,warc/revisit,5FWDJGLFCVXIOZOQ46G4WI5MUUP5FCF5,917
3686162,2023-09-25 22:44:19+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!20170228_Sewer_Asbuilt_ToSRC.xlsx/e7d90f47-786c-4860-910f-41329602057d,200.0,application/octet-stream,5FWDJGLFCVXIOZOQ46G4WI5MUUP5FCF5,15824
3686163,2023-09-26 20:32:37+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/!FilesIndex_5_6_2019.pdf/cb3dbea2-4c77-46d4-927f-bdc89db2e55b,200.0,application/octet-stream,OL5NQUXHN7LBPIOOV2BTBZPNM6LS2XH4,77723
...,...,...,...,...,...,...
5491403,2023-09-24 12:10:16+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web8272_h.pdf/59be36d5-29c1-4da9-8eb1-2ab82c1f6fe9,200.0,application/octet-stream,TZI3Z52KGNLIM6RSYVZ6E6CRO3VVNRH7,47259
5491404,2023-09-24 17:53:01+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web87.pdf/1e1bb4d9-ca7a-40dd-b3ca-0ffc0112c6b0,200.0,application/octet-stream,BPMHEL6OUDYVUHTQPOSJMK3SNRBYY2HF,98558
5491405,2023-09-24 12:10:56+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web8808_h.pdf/a07bcbb6-9c0e-4753-9cb6-0b62ba0843fa,200.0,application/octet-stream,U7BX7SQRR53QTSIPVPJCARKQ2XZ2XWXP,37998
5491406,2023-09-24 12:09:15+00:00,https://foiaonline.gov/foiaonline/api/request/downloadFile/~web9324_h.pdf/7d7a0524-6c07-4ef7-a044-6baab88f80da,200.0,application/octet-stream,JJJJMG6RL7Z4UCFBIU7MU5XXTCPZY7TH,46414


In [165]:
len(pdf_snaps.url.unique())

1551866

We only have references to 320,432 of them.